In [ ]:
# Import

from src.tag.src.text2sqlchain import generate_sql
from src.tag.src.query_executor import execute_text2sql_response
from src.tag.src.answer_generator import generate_answer
from src.tag.database.db_connection import connect_db
from src.tag.database.schema_loader import load_schema
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Connect ke database
conn = connect_db()

# Load schema database
schema = load_schema(conn)

d:\code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\code\src\tag\src\text2sqlchain.py:104: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_chain_zero = LLMChain(llm=llm, prompt=POSTGRES_PROMPT_ID)


In [ ]:
# Gemini API

# 📦 Import API KEY

import requests

# Ganti dengan API key kamu
api_key = os.getenv("GEMINI_API_TOKEN")

# Coba call endpoint Gemini simple
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={api_key}"

headers = {
    "Content-Type": "application/json",
}

body = {
    "contents": [
        {
            "parts": [
                {"text": "Halo, bisa dengar saya?"}
            ]
        }
    ]
}

# Kirim request
response = requests.post(url, headers=headers, json=body)

print("Status Code:", response.status_code)
print("Response Body:", response.text)


In [2]:
# 📋 Daftar pertanyaan untuk batch testing
# (Kosong dulu, kamu isi sendiri nanti di bawah ini)

questions1 = [
  "Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) yang digunakan?",
  "Apakah Lembaga Penyiaran Asing boleh mendirikan stasiun penyiaran di Indonesia?",
  "Bagaimana cara mendapatkan rekomendasi untuk menyelenggarakan Diklat Radio Elektronika atau Operator Radio (REOR)?",
  "Apa itu Jaringan Dokumentasi dan Informasi Hukum (JDIH) Kemkominfo, dan apa tujuannya?",
  "Bagaimana proses pendaftaran Nama Domain dilakukan, dan apa prinsip yang digunakan?",
  "Bagaimana ketentuan mengenai transparansi tarif layanan roaming internasional?",
  "Apa persyaratan teknis yang harus dipenuhi oleh pembaca kartu cerdas nirkontak?",
  "Bagaimana cara menyampaikan laporan pelanggaran hak cipta secara resmi?",
  "Bagaimana prosedur untuk mengajukan permohonan Nomor Protokol Internet?",
  "Apa yang dimaksud dengan alat dan perangkat telekomunikasi jarak dekat (short range devices), dan mengapa persyaratan teknisnya perlu diatur?",
]


print(f"Total Pertanyaan Diisi: {len(questions1)}")


Total Pertanyaan Diisi: 10


In [3]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions1, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch1.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch1.csv'")



==== Memproses Pertanyaan 1: Dalam pembangunan infrastruktur telekomunikasi, bagaimana cara perhitungan persentase TKDN untuk belanja modal atau capital expenditure (Capex) yang digunakan? ====


d:\code\src\tag\src\text2sqlchain.py:103: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = sql_chain.run(inputs)



==== Memproses Pertanyaan 2: Apakah Lembaga Penyiaran Asing boleh mendirikan stasiun penyiaran di Indonesia? ====

==== Memproses Pertanyaan 3: Bagaimana cara mendapatkan rekomendasi untuk menyelenggarakan Diklat Radio Elektronika atau Operator Radio (REOR)? ====

==== Memproses Pertanyaan 4: Apa itu Jaringan Dokumentasi dan Informasi Hukum (JDIH) Kemkominfo, dan apa tujuannya? ====

==== Memproses Pertanyaan 5: Bagaimana proses pendaftaran Nama Domain dilakukan, dan apa prinsip yang digunakan? ====

==== Memproses Pertanyaan 6: Bagaimana ketentuan mengenai transparansi tarif layanan roaming internasional? ====

==== Memproses Pertanyaan 7: Apa persyaratan teknis yang harus dipenuhi oleh pembaca kartu cerdas nirkontak? ====

==== Memproses Pertanyaan 8: Bagaimana cara menyampaikan laporan pelanggaran hak cipta secara resmi? ====

==== Memproses Pertanyaan 9: Bagaimana prosedur untuk mengajukan permohonan Nomor Protokol Internet? ====


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 36
}
].


⚠️ Kena rate limit (429). Menunggu 60 detik... [Percobaan ke-1]

==== Memproses Pertanyaan 10: Apa yang dimaksud dengan alat dan perangkat telekomunikasi jarak dekat (short range devices), dan mengapa persyaratan teknisnya perlu diatur? ====

✅ Semua hasil disimpan ke 'testing_text2sql_batch1.csv'


In [ ]:
#Batch 2 11-20
questions2 = [
  "Apa sanksi yang akan diberikan jika ada pihak yang melanggar ketentuan dalam peraturan Nomor Panggilan Darurat, misalnya memberikan informasi yang tidak benar saat melakukan panggilan ke Pusat Panggilan Darurat?",
  "Apa yang dimaksud dengan pemetaan urusan pemerintahan daerah di bidang komunikasi dan informatika dan mengapa hal ini penting?",
  "Apa saja tugas dan fungsi Dinas Komunikasi dan Informatika Provinsi dan Kabupaten atau Kota?",
  "Apa yang dapat saya lakukan jika saya merasa bahwa Data Pribadi saya telah disalahgunakan oleh Penyelenggara Sistem Elektronik (PSE)?",
  "Apa saja jenis layanan yang termasuk dalam ruang lingkup IPTV?",
  "Apa hak dan kewajiban lembaga pelatihan setelah terakreditasi?",
  "Apa itu Badan Aksesibilitas Telekomunikasi dan Informasi (Bakti), dan apa tugas utamanya?",
  "Bagaimana mekanisme pengusulan program KPU Telekomunikasi dan Informatika dari masyarakat atau pemerintah daerah?",
  "Apa perbedaan mendasar antara Tanda Tangan Elektronik dan Sertifikat Elektronik, dan bagaimana keduanya saling terkait?",
  "Jenis alat atau perangkat telekomunikasi apa saja yang wajib memenuhi persyaratan teknis?",
]  

print(f"Total Pertanyaan Diisi: {len(questions2)}")

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions2, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch2.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch2.csv'")


In [4]:
# Batch 3 
questions3 = [
 "Apa sanksi yang akan diberikan jika pemegang IPFR tidak menyampaikan laporan penggunaan Pita Frekuensi Radio secara berkala?",
  "Apa yang terjadi jika suatu handphone tidak memiliki nomor IMEI yang terverifikasi secara resmi?",
  "Apakah ada peraturan yang menjelaskan bahwa Penyelenggara Jasa Telekomunikasi wajib melakukan sosialisasi skema tarif yang jelas kepada pengguna atau pelanggan?",
  "Apakah registrasi kartu SIM (Nomor MSISDN) merupakan suatu kewajiban bagi pelanggan jasa telekomunikasi, dan apa konsekuensi hukum apabila kewajiban tersebut tidak dipenuhi?",
  "Apakah terdapat batasan jumlah nomor kartu SIM (Nomor MSISDN) yang dapat diregistrasi menggunakan satu identitas pelanggan?",
  "Apa saja yang menjadi fokus pemeriksaan dalam Audit TIK terhadap penerapan tata kelola dan manajemen TIK di SPBE?",
  "Apa saja yang harus diperhatikan dalam pengelolaan Nama Domain Pemerintah Daerah dan subdomain?",
  "Kenapa pemberian perizinan penggunaan spektrum frekuensi radio harus melalui proses seleksi?",
  "Apa yang harus diperhatikan oleh Penyelenggara Sistem Elektronik dalam hal melakukan pengelolaan, pemrosesan, atau penyimpanan Data Pribadi Pengguna?",
  "Bagaimana peran Pemerintah dalam memfasilitasi pemanfaatan Teknologi Informasi dan Transaksi Elektronik, serta melindungi kepentingan umum dari penyalahgunaan Informasi Elektronik?",
]

print(f"Total Pertanyaan Diisi: {len(questions3)}")

Total Pertanyaan Diisi: 10


In [5]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions3, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch3.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch3.csv'")



==== Memproses Pertanyaan 1: Apa sanksi yang akan diberikan jika pemegang IPFR tidak menyampaikan laporan penggunaan Pita Frekuensi Radio secara berkala? ====

==== Memproses Pertanyaan 2: Apa yang terjadi jika suatu handphone tidak memiliki nomor IMEI yang terverifikasi secara resmi? ====

==== Memproses Pertanyaan 3: Apakah ada peraturan yang menjelaskan bahwa Penyelenggara Jasa Telekomunikasi wajib melakukan sosialisasi skema tarif yang jelas kepada pengguna atau pelanggan? ====

==== Memproses Pertanyaan 4: Apakah registrasi kartu SIM (Nomor MSISDN) merupakan suatu kewajiban bagi pelanggan jasa telekomunikasi, dan apa konsekuensi hukum apabila kewajiban tersebut tidak dipenuhi? ====

==== Memproses Pertanyaan 5: Apakah terdapat batasan jumlah nomor kartu SIM (Nomor MSISDN) yang dapat diregistrasi menggunakan satu identitas pelanggan? ====

==== Memproses Pertanyaan 6: Apa saja yang menjadi fokus pemeriksaan dalam Audit TIK terhadap penerapan tata kelola dan manajemen TIK di SPBE? 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 4
}
].


⚠️ Kena rate limit (429). Menunggu 60 detik... [Percobaan ke-1]

==== Memproses Pertanyaan 10: Bagaimana peran Pemerintah dalam memfasilitasi pemanfaatan Teknologi Informasi dan Transaksi Elektronik, serta melindungi kepentingan umum dari penyalahgunaan Informasi Elektronik? ====

✅ Semua hasil disimpan ke 'testing_text2sql_batch3.csv'


In [ ]:
# Batch 4
questions4 = [
  "Apa kewajiban pelaku usaha ketika mereka akan melakukan usaha perdagangan melalui sistem elektronik? Terutama terkait dengan informasi produk dan perlindungan konsumen",
  "Apa saja asas, fungsi, dan peranan pers nasional?",
  "Tindakan apa saja yang dilarang dalam penggunaan Data Pribadi, dan sanksi pidana apa yang dapat dikenakan jika melanggar larangan tersebut?",
  "Apa saja yang menjadi hak dan kewajiban pelaksana penyediaan KPU Telekomunikasi?",
  "Apakah terdapat kewajiban bagi Pelaksana Penyedia Kewajiban Pelayanan Universal (KPU) Telekomunikasi untuk menggunakan produk dalam negeri dalam penyelenggaraan layanan telekomunikasi?",
  "Apa saja kewajiban yang dimiliki oleh perusahaan penyelenggara telekomunikasi, seperti Telkom dan Indosat, terkait dengan pengamanan jaringan internet yang mereka kelola?",
  "Ketika menggunakan layanan internet di rumah, data apa saja yang dicatat oleh penyedia jasa layanan internet (ISP) seperti Telkom atau penyedia lainnya?",
  "Apabila seseorang memanfaatkan jaringan WiFi yang disediakan secara publik, misalnya di kafe atau warnet, apakah data pribadi yang bersangkutan turut terekam oleh penyedia layanan WiFi tersebut?",
  "Terkait pengamanan jaringan internet. Apa sebenarnya yang ingin diamankan dan ancaman apa saja yang perlu diwaspadai?",
  "Apa itu Internet Exchange Point dan apa kewajibannya untuk menjaga keamanan internet?",
]

print(f"Total Pertanyaan Diisi: {len(questions4)}")

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions4, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch4.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch4.csv'")


In [ ]:
# Batch 5
questions5 = [
  "Apa itu Uji Laik Operasi (ULO) dan kapan penyelenggara jaringan telekomunikasi harus melaksanakannya?",
  "Apa bedanya proses seleksi dan evaluasi untuk mendapatkan izin penyelenggaraan jaringan telekomunikasi?",
  "Apakah izin penyelenggaraan jaringan telekomunikasi berlaku selamanya?",
  "Apa yang dimaksud dengan Layanan Konten SMS Premium dan apa saja kewajiban penyelenggaranya?",
  "Apakah benar siaran televisi analog akan diberhentikan, dan jika benar apa yang akan menggantikannya?",
  "Apa itu PSE Lingkup Privat dan siapa saja yang termasuk di dalamnya?",
  "Apa yang akan terjadi jika suatu PSE Lingkup Privat tidak melakukan pendaftaran ke kementerian?",
  "Ke daerah mana saja cakupan wilayah Siaran yang diperbolehkan dalam penyelenggaraan Penyiaran?",
  "Bagaimana cara mendapatkan IPP untuk penyelenggaraan Penyiaran?",
  "Apakah Informasi Elektronik bisa dijadikan alat bukti yang sah di pengadilan?",
]

print(f"Total Pertanyaan Diisi: {len(questions5)}")  

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions5, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch5.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch5.csv'")


In [ ]:
# Batch 6
questions6 = [
  "Apa saja perbuatan yang dilarang dalam UU Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik?",
  "Apa saja sanksi pidana yang bisa dikenakan jika melanggar UU Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik??",
  "Apakah ada kewajiban bagi Penyelenggara Sistem Elektronik untuk melindungi anak-anak yang menggunakan layanan mereka?",
  "Apa yang dimaksud dengan 'intersepsi' atau 'penyadapan' dalam UU Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, dan apakah semua tindakan penyadapan dilarang?",
  "Apa saja kewajiban Penyelenggara Sistem Elektronik?",
  "Apa judul dari Peraturan Pemerintah (PP) Nomor 80 Tahun 2019?",
  "Apa isi Pasal 2 Peraturan Pemerintah (PP) Nomor 80 Tahun 2019?",
  "Apa isi Pasal 10 Peraturan Menteri Komunikasi dan Informatika (PERMENKOMINFO) Nomor 4 Tahun 2016?",
  "Berdasarkan seluruh peraturan teknologi informasi yang ada di database, apa saja daftar seluruh judul subjek yang ada?",
  "Apa judul pasal selanjutnya setelah Pasal 39 UU Nomor 11 Tahun 2008?",
]

print(f"Total Pertanyaan Diisi: {len(questions6)}") 

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions6, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch6.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch6.csv'")


In [ ]:
# Batch 7
questions7 = [
 "Apa judul pasal sebelum Pasal 41 UU Nomor 11 Tahun 2008?",
  "Apakah Pasal 40 UU Nomor 11 Tahun 2008 pernah diamandemen? Dan jika pernah, apa judul pasal amandemennya?",
  "Apa urutan judul peraturan amandemen yang terjadi pada Peraturan Menteri Komunikasi dan Informatika Nomor 26 Tahun 2007?",
  "Apa pertimbangan atau latar belakang yang mendasari pembuatan PERMENKOMINFO Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat?",
  "Apa dasar hukum yang mendasari pembuatan PERMENKOMINFO Nomor 10 Tahun 2016 tentang Layanan Nomor Tunggal Panggilan Darurat?",
  "Apa saja judul pasal yang dirujuk oleh Pasal 25 PERMENKOMINFO No. 4 Tahun 2016?",
  "Apa saja judul pasal di peraturan lain yang mirip dengan Pasal 46 PP No. 52 Tahun 2000?",
  "Pasal 19 dalam Peraturan Kominfo Nomor 13 Tahun 2019 saat ini masih berlaku atau sudah dicabut?",
  "Apa judul Peraturan Pemerintah (PP) yang dikeluarkan pada tahun 2019 yang terkait dengan subjek Telekomunikasi, Informatika, Siber, dan Internet?",
  "Apa judul peraturan yang dikeluarkan pada rentang tahun 2008 hingga 2010 yang memiliki subjek Telekomunikasi, Informatika, Siber, dan Internet?",
]

print(f"Total Pertanyaan Diisi: {len(questions7)}") 

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions7, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch7.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch7.csv'")


In [ ]:
# Batch 8
questions8 = [
  "Pasal nomor berapa saja dari PERMENKOMINFO Nomor 26 Tahun 2007 yang sudah tidak berlaku?",
  "Pasal nomor berapa saja dari UU Nomor 11 Tahun 2008 yang sudah tidak berlaku?",
  "Berapa total jumlah pasal yang merujuk ke Pasal 57 atau dirujuk oleh Pasal 57 dalam Undang-Undang Nomor 27 Tahun 2022?",
  "PERMENKOMINFO No. 1 Tahun 2010 Pasal 70 sudah diamandemen berapa kali dan apa isi dari semua pasal amandemennya?",
  "Judul peraturan apa saja (lima) yang memiliki banyak pasal yang sudah tidak efektif atau tidak berlaku?",
  "Judul pasal apa saja yang berhubungan dengan Pasal 28 UU Nomor 11 Tahun 2008 dan apa jenis hubungannya?",
  "Apa saja daftar nama definisi yang diatur dalam Pasal 1 Peraturan Pemerintah (PP) Nomor 80 Tahun 2019 tentang Perdagangan Melalui Sistem Elektronik?",
  "Apa saja judul pasal yang paling banyak merujuk ke pasal lain atau dirujuk oleh pasal lain?",
  "Berdasarkan daftar peraturan yang ada di database, apa lima nama subjek yang paling banyak dibahas?",
  "Judul peraturan mana saja (lima) yang paling berpengaruh dalam ekosistem peraturan teknologi informasi Indonesia, berdasarkan banyaknya konsep atau aturan lain yang terhubung padanya?",
]

print(f"Total Pertanyaan Diisi: {len(questions8)}")  

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions8, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch8.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch8.csv'")


In [ ]:
# Batch 9
questions9 = [
  "Judul pasal mana saja (lima) yang paling berpengaruh dalam ekosistem peraturan teknologi informasi Indonesia, berdasarkan banyaknya konsep atau aturan lain yang terhubung padanya?",
  "Judul pasal mana saja (sepuluh) yang merupakan 'jembatan' antara kelompok pasal yang berbeda?",
  "Jika saya tertarik dengan Pasal 26 UU Nomor 19 Tahun 2016, judul pasal-pasal rekomendasi apa saja (sepuluh) yang sebaiknya saya baca?",
  "Kelompok peraturan mana saja yang secara erat terkait satu sama lain dan membentuk 'komunitas' regulasi yang berbeda?",
  "Apa jalur hubungan terdekat antara Pasal 26 UU Nomor 19 Tahun 2016 dengan Pasal 9 UU Nomor 27 Tahun 2022?",
  "Dari semua peraturan di bidang teknologi informasi di database, sebutkan semua pasal yang isinya membahas mengenai \"tanda tangan elektronik\", sebutkan judul pasalnya",
  "Dalam regulasi Nomor 27 Tahun 2022, sebutkan isi dari Pasal 27",
  "Peraturan bidang teknologi informasi apa saja (lima) yang paling banyak di amandemen oleh peraturan lain? Sebutkan judulnya",
  "Sebutkan semua judul peraturan di bidang teknologi informasi yang diterbitkan pada tahun 2016.",
  "Dari seluruh peraturan yang membahas \"sistem elektronik\", apa judul peraturan yang paling baru diterbitkan?",
]

print(f"Total Pertanyaan Diisi: {len(questions9)}")

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions9, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch9.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch9.csv'")


In [ ]:
# Batch 10
questions10 = [
  "Kapan (tanggal) Pasal 40 UU Nomor 11 Tahun 2008 terakhir kali diamandemen?",
  "Berapa banyak pasal yang terdapat di UU Nomor 27 Tahun 2022?",
  "Sebutkan 5 (lima) judul peraturan di bidang teknologi informasi terbaru yang diterbitkan oleh instansi \"Kementerian Komunikasi dan Informatika\"",
  "Pasal nomor berapa saja dari UU Nomor 11 Tahun 2008 yang sudah tidak berlaku setelah diamandemen oleh UU Nomor 19 Tahun 2016?",
  "Apa definisi \"Penyelenggaraan Jasa Telekomunikasi\" menurut PERMENKOMINFO Nomor 5 Tahun 2021?",
  "Apa hubungan atau relasi antara Pasal 35 dan Pasal 37 di UU nomor 11 tahun 2008?",
  "Apa hubungan atau relasi antara Pasal 40 UU Nomor 19 Tahun 2016 dengan Pasal 40 UU nomor 11 tahun 2008?",
  "Sebutkan semua judul peraturan yang mengatur tentang persyaratan teknis untuk telekomunikasi jarak dekat.",
  "Pasal nomor berapa saja dalam UU Nomor 11 Tahun 2008 yang berkaitan dengan ketentuan penyebaran informasi elektronik?",
  "Judul peraturan apa saja yang mengatur tentang penggunaan jaringan internet?"
]

print(f"Total Pertanyaan Diisi: {len(questions9)}")

In [ ]:
import time
import pandas as pd

results = []
MAX_RETRIES = 5
SLEEP_TIME = 60  # Selalu tunggu 60 detik saat kena rate limit

# Loop pertanyaan
for idx, question in enumerate(questions10, 1):
    print(f"\n==== Memproses Pertanyaan {idx}: {question} ====")
    retry_count = 0

    while retry_count < MAX_RETRIES:
        try:
            # 1. Generate SQL
            response = generate_sql(schema, question, top_k=100)

            # 2. Eksekusi SQL
            rows, columns = execute_text2sql_response(conn, response)

            # 3. Jawaban natural
            jawaban = generate_answer(columns, rows, question)

            # ✅ Sukses → simpan dan break
            
            results.append({
                "No": idx,
                "Question": question,
                # "Columns": columns,
                # "Rows": str(rows),
                "Answer": jawaban,
                "Status": "Sukses"
            })
            break  # keluar dari while → lanjut pertanyaan berikutnya

        except Exception as e:
            error_str = str(e)
            if "429" in error_str:
                print(f"⚠️ Kena rate limit (429). Menunggu {SLEEP_TIME} detik... [Percobaan ke-{retry_count+1}]")
                time.sleep(SLEEP_TIME)
                retry_count += 1
            else:
                print(f"❌ Error bukan karena limit: {e}")
                results.append({
                    "No": idx,
                    "Question": question,
                    "Answer": f"Error: {e}",
                    "Status": "Error"
                })
                break  # error fatal, lanjut ke pertanyaan berikutnya

    if retry_count == MAX_RETRIES:
        print(f"❌ Gagal terus karena rate limit. Lewatkan pertanyaan {idx}")
        results.append({
            "No": idx,
            "Question": question,
            "Answer": f"Error: Melebihi batas retry (429)",
            "Status": "Error"
        })

# Simpan hasil
df = pd.DataFrame(results)
df.to_csv("testing_text2sql_batch10.csv", index=False)
print("\n✅ Semua hasil disimpan ke 'testing_text2sql_batch10.csv'")
